# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={SQL Server Native Client 11.0};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

,Date,id,Warehouse,Shipments,Customer,ShipmentStatus
0,Jun 27 2022 8:30AM,240449,10,0085799125,ISDIN Corporation,Released
1,Jun 27 2022 8:30AM,240449,10,0085799129,ISDIN Corporation,Released
2,Jun 27 2022 8:30AM,240449,10,0085799128,ISDIN Corporation,Released
3,Jun 27 2022 8:30AM,240449,10,0085799133,ISDIN Corporation,Released
4,Jun 27 2022 8:30AM,240449,10,0085799153,ISDIN Corporation,Released
5,Jun 27 2022 8:30AM,240449,10,0085799152,ISDIN Corporation,Released
6,Jun 27 2022 8:30AM,240449,10,0085799158,ISDIN Corporation,Released
7,Jun 27 2022 8:30AM,240449,10,0085799159,ISDIN Corporation,Released
8,Jun 27 2022 8:30AM,240449,10,0085799162,ISDIN Corporation,Released
9,Jun 27 2022 8:30AM,240449,10,0085799164,ISDIN Corporation,Released


In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

,id,ShipmentStatus,Shipments
6,240442,Released,5
7,240446,Released,1
8,240447,Executing,5
9,240447,Released,3
10,240449,Released,48


In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus,Completed,Executing,Released
id,,,
240429,2.0,2.0,1.0
240442,NaN,NaN,5.0
240446,NaN,NaN,1.0
240447,NaN,5.0,3.0
240449,NaN,NaN,48.0


In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
240423,0.0,0.0,20.0
240424,0.0,26.0,28.0
240429,2.0,2.0,1.0
240442,0.0,0.0,5.0
240446,0.0,0.0,1.0


In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
240423,0,0,20
240424,0,26,28
240429,2,2,1
240442,0,0,5
240446,0,0,1


In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,240423,0,0,20
1,240424,0,26,28
2,240429,2,2,1
3,240442,0,0,5
4,240446,0,0,1


In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,240423,,,20
1,240424,,26,28
2,240429,2,2,1
3,240442,,,5
4,240446,,,1


In [11]:
shipment_pivot.dtypes

ShipmentStatus
id            int64
Completed    object
Executing    object
Released      int32
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

,Date,id,Warehouse,Customer
0,Jun 27 2022 8:30AM,240449,10,ISDIN Corporation
48,Jun 24 2022 4:06PM,240447,19,"AdvaGen Pharma, Ltd"
56,Jun 24 2022 3:47PM,240446,19,"GUSA Granules USA, Inc."
57,Jun 24 2022 3:39PM,240442,19,"GUSA Granules USA, Inc."
62,Jun 24 2022 3:00PM,240424,10,ISDIN Corporation
116,Jun 24 2022 2:59PM,240423,20,Alumier Labs Inc.
136,Jun 24 2022 2:54PM,240429,10,ISDIN Corporation


In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

,Date,id,Warehouse,Customer,Completed,Executing,Released
0,Jun 27 2022 8:30AM,240449,10,ISDIN Corporation,,,48
1,Jun 24 2022 4:06PM,240447,19,"AdvaGen Pharma, Ltd",,5,3
2,Jun 24 2022 3:47PM,240446,19,"GUSA Granules USA, Inc.",,,1
3,Jun 24 2022 3:39PM,240442,19,"GUSA Granules USA, Inc.",,,5
4,Jun 24 2022 3:00PM,240424,10,ISDIN Corporation,,26,28
5,Jun 24 2022 2:59PM,240423,20,Alumier Labs Inc.,,,20
6,Jun 24 2022 2:54PM,240429,10,ISDIN Corporation,2,2,1


In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Jun 27 2022 8:30AM,240449,10,ISDIN Corporation,48,,
1,Jun 24 2022 4:06PM,240447,19,"AdvaGen Pharma, Ltd",3,5,
2,Jun 24 2022 3:47PM,240446,19,"GUSA Granules USA, Inc.",1,,
3,Jun 24 2022 3:39PM,240442,19,"GUSA Granules USA, Inc.",5,,
4,Jun 24 2022 3:00PM,240424,10,ISDIN Corporation,28,26,
5,Jun 24 2022 2:59PM,240423,20,Alumier Labs Inc.,20,,
6,Jun 24 2022 2:54PM,240429,10,ISDIN Corporation,1,2,2


In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Jun 27 2022 8:30AM,240449,10,ISDIN Corporation,48,,
1,Jun 24 2022 4:06PM,240447,19,"AdvaGen Pharma, Ltd",3,5,
2,Jun 24 2022 3:47PM,240446,19,"GUSA Granules USA, Inc.",1,,
3,Jun 24 2022 3:39PM,240442,19,"GUSA Granules USA, Inc.",5,,
4,Jun 24 2022 3:00PM,240424,10,ISDIN Corporation,28,26,


In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Jun 27 2022 8:30AM,240449,10,ISDIN Corporation,48,NaN,NaN
1,Jun 24 2022 4:06PM,240447,19,"AdvaGen Pharma, Ltd",3,5.0,NaN
2,Jun 24 2022 3:47PM,240446,19,"GUSA Granules USA, Inc.",1,NaN,NaN
3,Jun 24 2022 3:39PM,240442,19,"GUSA Granules USA, Inc.",5,NaN,NaN
4,Jun 24 2022 3:00PM,240424,10,ISDIN Corporation,28,26.0,NaN


In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Jun 27 2022 8:30AM,240449,10,ISDIN Corporation,48,0.0,0.0
1,Jun 24 2022 4:06PM,240447,19,"AdvaGen Pharma, Ltd",3,5.0,0.0
2,Jun 24 2022 3:47PM,240446,19,"GUSA Granules USA, Inc.",1,0.0,0.0
3,Jun 24 2022 3:39PM,240442,19,"GUSA Granules USA, Inc.",5,0.0,0.0
4,Jun 24 2022 3:00PM,240424,10,ISDIN Corporation,28,26.0,0.0


In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

,id,Released,Executing,Completed
Warehouse,,,,
10,721302,77,28.0,2.0
19,721335,9,5.0,0.0
20,240423,20,0.0,0.0


In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

,Warehouse,id,Released,Executing,Completed
0,10,721302,77,28.0,2.0
1,19,721335,9,5.0,0.0
2,20,240423,20,0.0,0.0


In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

,Warehouse,Released,Executing,Completed
0,10,77,28.0,2.0
1,19,9,5.0,0.0
2,20,20,0.0,0.0


In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

,Warehouse,Status,Value
0,10,Released,77.0
1,19,Released,9.0
2,20,Released,20.0
3,10,Executing,28.0
4,19,Executing,5.0


In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse,10,19,20
Status,,,
Completed,2.0,0.0,0.0
Executing,28.0,5.0,0.0
Released,77.0,9.0,20.0


In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse,Status,10,19,20
0,Completed,2.0,0.0,0.0
1,Executing,28.0,5.0,0.0
2,Released,77.0,9.0,20.0


In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

,Status,10,19,20
0,Completed,2.0,0.0,0.0
1,Executing,28.0,5.0,0.0
2,Released,77.0,9.0,20.0


In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()